In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import flatten

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
tf.contrib.learn.python.learn.datasets.mnist.load_mnist()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbcf423b080>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbd48046d68>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fbcd41c7dd8>)

In [36]:
EPOCHS = 20
BATCH_SIZE = 50

In [ ]:
def conv2d(x, W, b, s=1):
    x = tf.nn.conv2d(x, W, strides=[1,s,s,1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1,k,k,1],
        strides=[1,k,k,1],
        padding='VALID')

def fully_connected(x, W, b):
    a = tf.add(tf.matmul(x, W), b)
    return tf.nn.tanh(a)
    
def LeNet(x):
    x = tf.reshape(x, (-1, 28, 28, 1))
    x = tf.pad(x, [[0,0], [2,2], [2,2], [0,0]], mode='CONSTANT')
    
    n_classes = 10
    
    layer_depth = {
        'L1':6,
        'L2':16,
        'FC':120
    }
    
    weights = {
        'L1':tf.Variable(tf.truncated_normal(
             [5,5,1,layer_depth['L1']])),
        'L2':tf.Variable(tf.truncated_normal(
             [3,3,layer_depth['L1'],layer_depth['L2']])),
        'FC':tf.Variable(tf.truncated_normal(
             [1024, layer_depth['FC']])),
        'OUT':tf.Variable(tf.truncated_normal(
             [layer_depth['FC'], n_classes]))
    }
    
    biases = {
        'L1':tf.Variable(tf.zeros(layer_depth['L1'])),
        'L2':tf.Variable(tf.zeros(layer_depth['L2'])),
        'FC':tf.Variable(tf.zeros(layer_depth['FC'])),
        'OUT':tf.Variable(tf.zeros(n_classes))
    }
    
    # Create layers
    conv1 = conv2d(x, weights['L1'], biases['L1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = conv2d(conv1, weights['L2'], biases['L2'])
    conv2 = maxpool2d(conv2)
    
    flat = flatten(conv2)
    
    fc_shape = flat.get_shape().as_list()[-1]
    weights['FC'] = tf.Variable(tf.truncated_normal([fc_shape, layer_depth['FC']]))
    
    fc = fully_connected(flat, weights['FC'], biases['FC'])
    
    out = fully_connected(fc, weights['OUT'], biases['OUT'])
    
    return out
    

In [37]:
from models import ConvNet

def LeNet(x, keep_prob):
    x = tf.reshape(x, (-1, 28, 28, 1))
    x = tf.pad(x, [[0,0], [2,2], [2,2], [0,0]], mode='CONSTANT')
    
    model = ConvNet(training_epochs=10, batch_size=50, n_classes=10)
    model.DATA = x
    model.KEEP_PROB = (keep_prob, 0.5)

    model.conv2d('L1', kernel_size=5, depth=6)
    model.pool2d('MAX')

    model.conv2d('L2', kernel_size=3, depth=16)
    model.pool2d('MAX')

    model.fully_connected('FC', depth=120)
    model.dropout()
    
    model.fully_connected('OUT', depth=10, ACTIVATION=None)
    return model.LOGITS

In [38]:
# MNIST consists of 28x28x1, grayscale images
x = tf.placeholder(tf.float32, (None, 784))
# Classify over 10 digits 0-9
y = tf.placeholder(tf.float32, (None, 10))
keep_prob = tf.placeholder(tf.float32)
fc2 = LeNet(x, keep_prob)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(fc2, y))
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op)
correct_prediction = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


def eval_data(dataset):
    """
    Given a dataset as input returns the loss and accuracy.
    """
    # If dataset.num_examples is not divisible by BATCH_SIZE
    # the remainder will be discarded.
    # Ex: If BATCH_SIZE is 64 and training set has 55000 examples
    # steps_per_epoch = 55000 // 64 = 859
    # num_examples = 859 * 64 = 54976
    #
    # So in that case we go over 54976 examples instead of 55000.
    steps_per_epoch = dataset.num_examples // BATCH_SIZE
    num_examples = steps_per_epoch * BATCH_SIZE
    total_acc, total_loss = 0, 0
    for step in range(steps_per_epoch):
        batch_x, batch_y = dataset.next_batch(BATCH_SIZE)
        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_acc += (acc * batch_x.shape[0])
        total_loss += (loss * batch_x.shape[0])
    return total_loss/num_examples, total_acc/num_examples


if __name__ == '__main__':
    # Load data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables()) # tf.global_variables_initializer())
        steps_per_epoch = mnist.train.num_examples // BATCH_SIZE
        num_examples = steps_per_epoch * BATCH_SIZE

        # Train model
        for i in range(EPOCHS):
            for step in range(steps_per_epoch):
                batch_x, batch_y = mnist.train.next_batch(BATCH_SIZE)
                loss = sess.run(train_op, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})

            val_loss, val_acc = eval_data(mnist.validation)
            print("EPOCH {} ...".format(i+1))
            print("Validation loss = {:.3f}".format(val_loss))
            print("Validation accuracy = {:.3f}".format(val_acc))
            print()

        # Evaluate on the test data
        test_loss, test_acc = eval_data(mnist.test)
        print("Test loss = {:.3f}".format(test_loss))
        print("Test accuracy = {:.3f}".format(test_acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
EPOCH 1 ...
Validation loss = 3.076
Validation accuracy = 0.233

EPOCH 2 ...
Validation loss = 2.260
Validation accuracy = 0.191

EPOCH 3 ...
Validation loss = 2.199
Validation accuracy = 0.191

EPOCH 4 ...
Validation loss = 2.085
Validation accuracy = 0.208

EPOCH 5 ...
Validation loss = 2.027
Validation accuracy = 0.230

EPOCH 6 ...
Validation loss = 1.923
Validation accuracy = 0.272

EPOCH 7 ...
Validation loss = 1.905
Validation accuracy = 0.356

EPOCH 8 ...
Validation loss = 1.828
Validation accuracy = 0.389

EPOCH 9 ...
Validation loss = 1.687
Validation accuracy = 0.358

EPOCH 10 ...
Validation loss = 1.558
Validation accuracy = 0.414

EPOCH 11 ...
Validation loss = 1.541
Validation accuracy = 0.417

EPOCH 12 ...
Validation loss = 1.536
Validation accuracy = 0.413

EPOCH 13 ...
Validation

In [ ]:
As an additional fun exercise calculate the total number of parameters used by the network. Note, the convolutional layers use weight sharing!
